<br>


<div align="left">Réalisé par : Mekie Chami Karl</div>
<div align="left">Encadré par : Mekie Ralph Kevin</div>
<div align="left">Année : 2025</div><br><br><br>

<div align="center">
  <span style="font-family:Lucida Calligraphy; font-size:30px;">
    Projet en finance de marché
  </span>
</div>
<br><br>
<hr>


<br>

<div align="center">
    <span style=" font-size:32px; color: #00008B;"> 
        Impact de la baisse des taux de la BCE sur un portefeuille d'obligations
    </span>
</div><br>
<hr>

<div align="center" style="border-bottom:solid pink">
    <h1>Introduction et présentation des données</h1>
</div>

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3> Problématique et présentation des données</h3>
</div>

##### <span style="color: green;">**Définition d'une Obligation**
</strong></span> 

Une obligation est un instrument financier représentant un emprunt contracté par un État, une entreprise ou toute entité auprès d’investisseurs. En achetant une obligation, l’investisseur prête de l’argent à **l’émetteur** en échange de paiements **d’intérêts réguliers (appelés coupons)** et du remboursement du **capital** à une date fixée (la maturité). Les obligations sont généralement considérées comme des placements moins risqués que les actions, mais leur rendement dépend de facteurs comme le **taux d’intérêt**, la qualité de crédit de l’émetteur et **l’inflation**.

Caractéristiques principales d’une obligation à savoir pour projet :

* Nominal (valeur faciale) : montant que l’émetteur rembourse à l’échéance.

* Prix d’émission : prix auquel l’obligation est vendue au départ.

* Prix de marché : prix actuel sur le marché secondaire.

* Taux nominal (ou facial) : taux fixé à l’émission pour calculer le coupon.

* Coupon : montant périodique versé à l’investisseur.

* Fréquence des coupons : annuel, semi-annuel, trimestriel, etc.

* Date d’émission : date à laquelle l’obligation est créée.

* Date de jouissance : date à partir de laquelle les intérêts commencent à courir.

* Date de maturité (échéance) : date de remboursement du capital.

* Durée de vie (tenor) : durée jusqu’à maturité.

* Taux actuariel (Yield to Maturity - YTM) : **rendement global** si l’obligation est conservée jusqu’à l’échéance.

* Rendement courant : coupon / prix de marché.

* Type d’obligation : **taux fixe, taux variable, zéro coupon, inflation,...**

* Base de calcul des intérêts : convention de jour (ex : Actual/360, 30/360…).


##### <span style="color: green;">**Problématique**
</strong></span> 

**Le prix d’une obligation** est influencé par de multiples facteurs de marché. L’objectif de ce rapport est de présenter des méthodes permettant de calculer le prix d’une obligation après un stress de marché, c’est-à-dire une perturbation affectant ces facteurs. Nous disposons donc d'un portefeuille obligataire composé d'obligation **zéro-coupon**, à **taux fixes**, à **taux variables** et indexées sur **l'inflation**. Deux approches principales seront détaillées :

1. **Méthode Delta-Gamma** : Cette méthode estime le nouveau prix d’une obligation en fonction de sa **duration** et de sa **convexité**, en supposant une **variation constante** du **yield to maturity** (YTM) pour chaque facteur de risque.

2. **Méthode du full repricing** : Cette approche recalcule intégralement le prix de l’obligation après un stress, qui peut être constant ou variable selon les différents facteurs de risque. Elle nécessite le calcul préalable du **z-spread**, une mesure du spread ajouté aux taux sans risque pour égaler le prix de marché. Cette méthode est plus précise mais plus lourde, car nécessitant plus de calculs.


##### <span style="color: green;">**Base Bonds**
</strong></span> 

Nous disposons d'une base d'obligation générée par la fonction présente dans le fiichier bond_fonction. \
Cette base est composée des variables ci-dessous :

* ISIN : International Securities Identification Number, c’est un code standardisé sur 12 caractères qui identifie de manière unique un instrument financier à l’échelle mondiale.
* accrual_per : durée de temps entre la date de jouissance et la date à laquelle on veut repricer l'obligation
* initial_TTM : durée initiale jusqu’à maturité.
* TTM : durée restante jusqu’à maturité.
* first_accrual_date : Date de jouissance
* first_coupon_date : Date du premier coupon
* maturity_date : date de remboursement du capital
* coupon_rate : pourcentage nu nominal correspondant au coupon (montant périodique versé à l’investisseur), il s'agit d'un taux annuel.
* prices : prix actuel sur le marché secondaire.
* freq : Fréquence des coupons
* bond_type : Type d’obligation (taux fixe, taux variable, zéro coupon, inflation)
* spread : pour une obligation à taux variable, spread à rajouter au taux de coupon.
* convention : Base de calcul des intérêts

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3> Import des packages, modules et choix des paramètres</h3>
</div>

In [90]:
import os as os
import numpy as np
import time as time
import pandas as pd
import datetime as dt
import dateutil as dtu
from scipy.interpolate import CubicSpline

In [91]:
import date_functions as dtf
import bond_generation_functions as bgf
#import SpecialInterpols as Interps #Importer le module nécésaire à l'interpolation cubique/qui peut etre remplacé par celui de spicy.interpolate

In [92]:
#Chemin
ProjectDir = r"C:\Users\DELL\anaconda_projects\Projet finalisé\MulticurveFrameWork_next_jupiyter"
os.chdir(ProjectDir)


In [93]:

#paramètres de dates
start_date_char = "30/06/2020"
globals()['computation_date'] = dt.datetime.strptime(start_date_char,"%d/%m/%Y")
computation_date = globals()['computation_date']


<div align="center" style="border-bottom:solid  pink">
<h1> 1 - Théorie du calcul d'un Z_spread</h1>
</div>

Le **z-spread** (ou spread de zéro-coupon) est une mesure utilisée pour évaluer le rendement supplémentaire qu'une obligation offre par rapport à une courbe de taux sans risque (généralement basée sur des obligations d'État). 

Nous aurons besoin d'un algorithme de résolution d'équation pour le calcul du Z_spread, nous proposons celui de Newton_Raphson que nous présentons à la prochaine sous-partie. 

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>1.0 - Algorithme de Newton-Raphson </h3>
</div>

Soit $f$ une fonction dérivable. On effectue un développement limité au premier ordre autour d’un point $x_0$ :

$$
f(x) \approx f(x_0) + f'(x_0)(x - x_0)
$$

Si on cherche une racine $x$ telle que $f(x) = 0$, alors :

$$
0 \approx f(x_0) + f'(x_0)(x - x_0)
$$

En isolant $x$, on obtient :

$$
x \approx x_0 - \frac{f(x_0)}{f'(x_0)}
$$

On pose $x_1 = x_0 - \frac{f(x_0)}{f'(x_0)}$. En répétant ce raisonnement, on construit une suite $(x_n)$ définie par :

$$
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
$$

C’est l’**algorithme de Newton-Raphson**, utilisé pour approximer une solution de l’équation $f(x) = 0$.

**Étapes de l'algorithme :**

1. Initialiser $x = x_0$ et $n = 0$
2. Tant que $|f(x)| > \varepsilon$ et $n < N_{\text{max}}$ :
    - Si $f'(x) = 0$ :
        - **Arrêter** : la dérivée est nulle, pas de convergence
    - Calculer la nouvelle approximation :
      $$
      x \leftarrow x - \frac{f(x)}{f'(x)}
      $$
    - Incrémenter $n \leftarrow n + 1$
3. Retourner $x$ comme estimation de la racine


<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>1.1 - Obligation Zéro-coupon </h3>
</div>

Pour une obligation zéro-coupon, qui ne verse pas de coupons intermédiaires, le calcul est simplifié car il n'y a qu'un seul flux de trésorerie à l'échéance. Voici une méthode pas à pas pour le déterminer.



<span style="color: black;">**Calcul du z-spread pour une obligation zéro-coupon à un temps $ t $**
</strong></span> 

 Nous supposons que nous sommes à un temps $ t $ entre $ t_{i-1} $ et $ t_i $, avant l’échéance finale, avec un seul paiement à venir. Voici la méthode adaptée.


Considérons une obligation zéro-coupon avec :
- Valeur nominale : $ F $ (payée à l’échéance)
- Prix de marché actuel à $ t $ : $ P $
- Temps actuel : $ t $

Le z-spread, noté $ s $, est la constante ajoutée au taux sans risque $ r(t_N) $ pour actualiser le flux final à partir de $ t $. Comme il s’agit d’une zéro-coupon, il n’y a pas de coupon couru.

<span style="color: green;">**Étape 2 : Modèle mathématique**
</strong></span> 

Le prix $ P $ est la valeur actualisée du flux final $ F $ à l’échéance $ t_N $, sans coupon couru :

$ P = \frac{F}{(1 + r(t_N) + s)^{\tau(t, t_N)}} $

où :
- $ F $ est le paiement final à l’échéance,
- $ r(t_N) $ est le taux sans risque à $ t_N $,
- $ s $ est le z-spread à déterminer,
- $ \tau(t, t_N) = t_N - t $ est le temps relatif entre $ t $ et l’échéance $ t_N $.

L’objectif est de résoudre cette équation pour $ s $.

<span style="color: green;">**Étape 3 : Résolution pour le z-spread**
</strong></span> 

Pour une zéro-coupon, une solution analytique est possible :
1. Réécrire l’équation :
   $ P (1 + r(t_N) + s)^{\tau(t, t_N)} = F $
2. Isoler le terme exponentiel :
   $ (1 + r(t_N) + s)^{\tau(t, t_N)} = \frac{F}{P} $
3. Prendre la racine $ \tau(t, t_N) $-ième :
   $ 1 + r(t_N) + s = \left( \frac{F}{P} \right)^{\frac{1}{\tau(t, t_N)}} $
4. Isoler $ s $ :
   $ s = \left( \frac{F}{P} \right)^{\frac{1}{\tau(t, t_N)}} - 1 - r(t_N) $

Alternativement, une itération numérique peut être utilisée pour comme celle présentée précedemment.

<span style="color: green;">**Remarque**
</strong></span> 

- Pas de coupon couru pour une zéro-coupon, donc $ P $ est uniquement le prix net.
- La solution analytique est privilégiée, mais une itération peut être utilisée pour confirmer.

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>1.2 - Obligation à taux fixe </h3>
</div>

<span style="color: black;">**Calcul du z-spread pour une obligation à taux fixe à un temps $ t $ entre $ t_{i-1} $ et $ t_i $ avec coupon couru**

</strong></span> 

Nous supposons que nous sommes à un temps $ t $ entre $ t_{i-1} $ et $ t_i $, après le paiement du coupon à $ t_{i-1} $ mais avant celui à $ t_i $. Voici la méthode ajustée.

Considérons une obligation à taux fixe avec : 
- Valeur nominale : $ F $
- Prix de marché actuel à $ t $ (prix total) : $ P $ (incluant le coupon couru)
- Taux de coupon annuel : $ c $
- Nombre total de périodes initial : $ N $ 
- Temps jusqu’à $ t_i $ : $ t_i - t $ (où $ 0 < t_i - t < 1 $ si les périodes sont normalisées à 1)
- Temps relatif à chaque flux : $ \tau(t, t_k) = t_k - t $ (pour $ t_k = t_i, t_{i+1}, ..., N $)
- Nombre de périodes restantes jusqu’à $ N $ : $ n = N - t_i + 1 $ (incluant $ t_i $)
- Taux sans risque pour chaque période $ t_k $ : $ r(t_k) $ (de $ t_i $ à $ N $)
- Fréquence des paiements : $ m $ 
- Coupon couru : $ CC = C \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} $, où $ C = \frac{c \cdot F}{m} $

Le z-spread, noté $ s $, est la constante ajoutée aux taux sans risque $ r(t_k) $ pour actualiser les flux futurs à partir de $ t $, en ajustant pour le coupon couru.

<span style="color: green;">**Modèle mathématique** 
</strong></span> 


$ P_{\text{Dirty}} = \sum_{k=t_i}^{N} \frac{C}{(1 + r(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r(N) + s)^{\tau(t, t_N)}} $


Ainsi, l’équation complète devient :

$ P_{\text{Clean}} = \sum_{k=t_i}^{N} \frac{C}{(1 + r(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r(N) + s)^{\tau(t, t_N)}} - C \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} $

L’objectif est de résoudre pour $ s $.

<span style="color: green;">**Résolution pour le z-spread** 
</strong></span> 

Appliquer l'algortihme de Newton-raphson sur l'équation $f(s)=0$
où $ f(s) = \sum_{k=t_i}^{N} \frac{C}{(1 + r(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r(N) + s)^{\tau(t, t_N)}} - C \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} - P_{\text{Clean}}$

$ f'(s) = \sum_{k=t_i}^{N} -\tau(t, t_k) \cdot \frac{C}{(1 + r(t_k) + s)^{\tau(t, t_k) + 1}} - \frac{\tau(t, t_N) \cdot F}{(1 + r(N) + s)^{\tau(t, t_N) + 1}} $

1. Choisir une valeur initiale $ s_0 $. 
2. Itérer avec la formule :  
   $ s_{n+1} = s_n - \frac{f(s_n)}{f'(s_n)} $,  
   où $ f(s_n) $ et $ f'(s_n) $ sont calculés à chaque étape.  
3. Répéter jusqu’à ce que $ |f(s_n)| < \epsilon $ (par exemple, $ \epsilon = 10^{-6} $) ou que $ s_n $ converge.

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>1.3 - Obligation indexée sur l'inflation </h3>
</div>

<span style="color: black;">**Calcul du z-spread pour une obligation indexée sur l’inflation à un temps $ t $ entre $ t_{i-1} $ et $ t_i $ avec coupon couru**
</strong></span> 
 
Nous supposons que nous sommes à un temps $ t $ entre $ t_{i-1} $ et $ t_i $, après le paiement du coupon à $ t_{i-1} $ mais avant celui à $ t_i $. Pour une obligation indexée sur l’inflation, les flux sont ajustés par l’inflation, et le taux réel est dérivé des courbes zéro-coupon (ZC) et break-even. Voici la méthode ajustée.

Considérons une obligation indexée sur l’inflation avec : 
- Valeur nominale : $ F $ (ajustée par l’inflation à l’échéance)
- Prix de marché actuel à $ t $ (prix total) : $ P $ (incluant le coupon couru)
- Taux de coupon réel annuel : $ c $ (payé sur le principal ajusté par l’inflation)
- Taux zéro-coupon (ZC) pour chaque période $ t_k $ : $ zc(t_k) $ (de $ t_i $ à $ N $, tiré de la courbe ZC)
- Taux d’inflation break-even pour chaque période $ t_k $ : $ be(t_k) $ (de $ t_i $ à $ N $, tiré de la courbe break-even)
- Taux réel pour chaque période $ t_k $ : $ r_{\text{réel}}(t_k) $, calculé par $ 1 + zc(t_k) = (1 + r_{\text{réel}}(t_k)) \cdot (1 + be(t_k)) $
- Fréquence des paiements : $ m $
- Coupon réel : $ C = \frac{c \cdot F}{m} $ (ajusté par l’inflation pour chaque période)
- Coupon couru : $ CC = C \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} $

Le z-spread, noté $ s $, est la constante ajoutée aux taux réels $ r_{\text{réel}}(t_k) $ pour actualiser les flux futurs à partir de $ t $, en ajustant pour le coupon couru.

<span style="color: green;">**Calcul du taux réel**
</strong></span> 

Pour chaque période $ t_k $, isolons $ r_{\text{réel}}(t_k) $ dans la relation donnée :
$ 1 + zc(t_k) = (1 + r_{\text{réel}}(t_k)) \cdot (1 + be(t_k)) $

$ r_{\text{réel}}(t_k) = \frac{1 + zc(t_k)}{1 + be(t_k)} - 1 $

Ce taux réel remplace le taux sans risque dans la formule de valorisation.

<span style="color: green;">**Modèle mathématique** 
</strong></span> 

$ P_{\text{Dirty}} = \sum_{k=t_i}^{N} \frac{C}{(1 + r_{\text{réel}}(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r_{\text{réel}}(N) + s)^{\tau(t, t_N)}} $


Ainsi, l’équation complète devient :

$ P_{\text{Clean}} = \sum_{k=t_i}^{N} \frac{C}{(1 + r_{\text{réel}}(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r_{\text{réel}}(N) + s)^{\tau(t, t_N)}} - C \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} $

L’objectif est de résoudre pour $ s $.

<span style="color: green;">**Résolution pour le z-spread** 
</strong></span> 

Appliquer l’algorithme de Newton-Raphson sur l’équation $ f(s) = 0 $, où :

$ f(s) = \sum_{k=t_i}^{N} \frac{C}{(1 + r_{\text{réel}}(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r_{\text{réel}}(N) + s)^{\tau(t, t_N)}} - C \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} - P_{\text{Clean}} $

Dérivée pour Newton-Raphson:
La dérivée $ f'(s) $ est nécessaire pour l’itération :
$ f'(s) = \sum_{k=t_i}^{N} \frac{-\tau(t, t_k) \cdot C}{(1 + r_{\text{réel}}(t_k) + s)^{\tau(t, t_k) + 1}} - \frac{\tau(t, t_N) \cdot F}{(1 + r_{\text{réel}}(N) + s)^{\tau(t, t_N) + 1}} $

1. Choisir une valeur initiale $ s_0 $. 
2. Itération : $ s_{n+1} = s_n - \frac{f(s_n)}{f'(s_n)} $, jusqu’à convergence.
3. Répéter jusqu’à ce que $ |f(s_n)| < \epsilon $ (par exemple, $ \epsilon = 10^{-6} $) ou que $ s_n $ converge.


<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>1.4 - Obligation à taux variable </h3>
</div>

**Obligations à taux variable**

Les obligations à taux variable (ou "floating rate notes", FRN) sont des titres de dette dont le taux de coupon n’est pas fixe, mais varie périodiquement en fonction d’un taux de référence, généralement un taux interbancaire comme le LIBOR, l’EURIBOR ou, plus récemment, le SOFR. Leur structure vise à réduire le risque de taux d’intérêt pour les investisseurs, car les paiements s’ajustent aux conditions du marché. Le coupon est typiquement défini comme la somme du taux de référence $ r_{\text{ref}}(t) $ (observé à une date de révision) et d’une marge fixe $ m $ (spread), soit $ c_t = r_{\text{ref}}(t) + m $. Cette marge reste constante sur la durée de l’obligation, tandis que le taux de référence est réévalué à intervalles réguliers (par exemple, tous les 3 ou 6 mois).

Le principal remboursé à l’échéance reste fixe, égal à la valeur nominale initiale $ F $, sauf disposition contraire. Les paiements de coupons, calculés comme $ C_t = \frac{c_t \cdot F}{m} $ (où $ m $ est la fréquence annuelle des paiements), fluctuent donc avec le taux de référence, rendant la valeur de l’obligation moins sensible aux variations des taux d’intérêt par rapport à une obligation à taux fixe. Ces obligations sont particulièrement attractives dans des environnements où les taux montent, car elles offrent une protection naturelle contre l’érosion de la valeur présente liée à une hausse des rendements exigés. Elles sont souvent émises par des entreprises ou des institutions financières.


<span style="color: black;">**Calcul du z-spread pour une obligation à taux variable indexée sur Euribor 6M à un temps $ t $ entre $ t_{i-1} $ et $ t_i $**
</strong></span>\
Nous supposons que nous sommes à un temps $ t $ entre $ t_{i-1} $ et $ t_i $, après le paiement du coupon à $ t_{i-1} $ mais avant celui à $ t_i $. L’obligation à taux variable est indexée sur l’Euribor 6M, avec un spread fixe $ s_{\text{spread}} $ ajouté au taux de coupon. 

Considérons une obligation à taux variable avec :
- Valeur nominale : $ F $
- Prix de marché actuel à $ t $ (prix total)** : $ P $ (incluant le coupon couru)
- Taux de coupon variable : $ c(t_k) = e(t_k) + s_{\text{spread}} $, où $ e(t_k) $ est le taux Euribor 6M applicable à $ t_k $, et $ s_{\text{spread}} $ est le spread fixe de l’obligation
- Fréquence des paiements : $ m = 2 $ (semestriel, aligné sur Euribor 6M)
- Taux Euribor 6M : 
  - Pour $ t_i $, le taux $ e(t_i) = e(t_{i-1}) $, fixé à $ t_{i-1} $ et obtenu sur le marché (car nous sommes à $ t > t_{i-1} $, le taux applicable à $ t_i $ est déjà connu il s'agit d'un taux déposit),
  - Pour $ t_k > t_i $, $ e(t_k) $ est le taux Euribor 6M forward implicite à $ t $, tiré d’une courbe forward.
- Coupon variable : $ C(t_k) = \frac{c(t_k) \cdot F}{m} = \frac{(e(t_k) + s_{\text{spread}}) \cdot F}{2} $
- Coupon couru : $ CC = C(t_i) \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} $, où $ C(t_i) = \frac{(e(t_{i-1}) + s_{\text{spread}}) \cdot F}{2} $

Le z-spread, noté $ s $, est la constante ajoutée aux taux sans risque $ r(t_k) $ pour actualiser les flux futurs à partir de $ t $, en ajustant pour le coupon couru.


<span style="color: green;">**Modèle mathématique**</strong></span>


$ P_{\text{Dirty}} = \sum_{k=t_i}^{N} \frac{C(t_k)}{(1 + r(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r(N) + s)^{\tau(t, t_N)}} $



$ P_{\text{Clean}} = \sum_{k=t_i}^{N} \frac{C(t_k)}{(1 + r(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r(N) + s)^{\tau(t, t_N)}} - C(t_i) \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} $


<span style="color: green;">**Résolution pour le z-spread**</strong></span>
 
Appliquer l’algorithme de Newton-Raphson sur l’équation $ f(s) = 0 $, où : 

$ f(s) = \sum_{k=t_i}^{N} \frac{C(t_k)}{(1 + r(t_k) + s)^{\tau(t, t_k)}} + \frac{F}{(1 + r(N) + s)^{\tau(t, t_N)}} - C(t_i) \cdot \frac{t - t_{i-1}}{t_i - t_{i-1}} - P_{\text{Clean}} $

La dérivée $ f'(s) $ est calculée comme suit :  


$ f'(s) = \sum_{k=t_i}^{N} -\tau(t, t_k) \cdot \frac{C(t_k)}{(1 + r(t_k) + s)^{\tau(t, t_k) + 1}} - \frac{\tau(t, t_N) \cdot F}{(1 + r(N) + s)^{\tau(t, t_N) + 1}} $

<span style="color: green;">**Algorithme de Newton-Raphson**</strong></span> 
 
1. Choisir une valeur initiale $ s_0 $   
2. Itérer avec la formule :  
   $ s_{n+1} = s_n - \frac{f(s_n)}{f'(s_n)} $,  
   où $ f(s_n) $ et $ f'(s_n) $ sont calculés à chaque étape.  
3. Répéter jusqu’à ce que $ |f(s_n)| < \epsilon $ (par exemple, $ \epsilon = 10^{-6} $) ou que $ s_n $ converge.

<span style="color: green;">**Remarque**</strong></span>

- Le taux Euribor 6M  observé fixé à $ t_{i-1} $ est utilisé pour $ t_i $, puis les taux forward pour les périodes suivantes.
- Le $ s_{\text{spread}} $ est distinct du $z_{spread}$ $ s $, ce dernier ajustant les taux sans risque.

<div align="center" style="border-bottom:solid  pink">
<h1> 2 - Calcul du z_spread et du YTM</h1>
</div>

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>2.1 - Prerécquis </h3>
</div>

In [111]:
np.random.seed(123)
#génération d'obligations
bond_df = bgf.generate_n_bonds(nb_bonds = 10, bnd_ctry='FR', computation_date_dt=computation_date)


In [112]:
bond_df.head()

,bond_id,ISIN,accrual_per,TTM,initial_TTM,first_accrual_date,first_coupon_date,maturity_date,coupon_rate,prices,freq,bond_type,spread,convention
0,0,FR0038622365,23.400833,18.599167,42,1997-02-05,NaT,2039-02-03,0.000000,117.577045,0,zero,NaN,ACT/ACT
1,1,FR0021099549,12.327731,13.672269,26,2008-03-03,2009-03-02,2034-03-02,1.869007,114.508332,1,infla,NaN,ACT/ACT
2,2,FR0095538145,17.390829,2.609171,20,2003-02-10,2004-02-07,2023-02-07,1.172564,113.254871,1,fix,NaN,ACT/ACT
3,3,FR0034895483,4.198523,21.801477,26,2016-04-19,2017-04-18,2042-04-18,4.939976,109.258794,1,fix,NaN,ACT/ACT
4,4,FR0014637825,12.030527,12.969473,25,2008-06-19,2008-12-20,2033-06-20,NaN,108.213897,2,float,0.01915,ACT/360


<div align="left" style="border-bottom: 3px solid green; width: 20%;">
    <h6> Courbe taux Zero coupon </h6>
</div>

In [114]:
DataDir = "\\".join([ProjectDir,'Data'])
#Courbe Zero coupon
zc_raw_df = pd.read_csv(DataDir+"\\FR_ZC_Inputs.csv", sep=',').rename(columns={'Value':'zc_rate'})
zc_raw_df.T


,0,1,2,3,4,5,6,7,8,9,10,11,12
ID,1,1,2,3,4,5,6,7,8,9,10,11,12
Tenor,1D,1M,3M,6M,1Y,2Y,3Y,5Y,7Y,10Y,20Y,30Y,50Y
zc_rate,2.26,2.26,2.28,2.26,2.21,2.16,2.5,2.69,3.0,3.45,3.84,4.08,4.08


A l'aide une fonction nous ajoutons les parametres de dates où alpha c'est $ \tau(spotdate,adjdate) $

In [116]:
#Ajout des dates
zc_df = dtf.add_dates(zc_raw_df, computation_date, computation_date, convention = "ACT/365")
zc_df.head()

,ID,Tenor,zc_rate,cmpt_date,spot_date,unadj_date,adj_date,day_diff,convention,alpha
0,1,1D,2.26,2020-06-30,2020-06-30,2020-07-01,2020-07-01,1,ACT/365,0.002740
1,1,1M,2.26,2020-06-30,2020-06-30,2020-07-30,2020-07-30,30,ACT/365,0.082192
2,2,3M,2.28,2020-06-30,2020-06-30,2020-09-30,2020-09-30,92,ACT/365,0.252055
3,3,6M,2.26,2020-06-30,2020-06-30,2020-12-30,2020-12-30,183,ACT/365,0.501370
4,4,1Y,2.21,2020-06-30,2020-06-30,2021-06-30,2021-06-30,365,ACT/365,1.000000


<div align="left" style="border-bottom: 3px solid green; width: 20%;">
    <h6> Courbe taux réel</h6>
</div>

sachant que nous n'avons pas accès à Bloomberg nous allons génerer aléatoirement l'inflation break even nécessaire pour calculer le taux réel

In [119]:
# As we don't have Bloomberg acces for inflation_swap, we will generate it randomly -- assumed as  inflation beakeven pages 25 to 34 here "https://cnofrance.org/wp-content/uploads/2021/05/2011_07_15-Normes-Usages-Indexed-Bonds.pdf" 
zc_df['inf_swp_rate'] = np.random.uniform(1,3,zc_df.shape[0])


En utilisant la formule de fisher nous déduisons le taux réel
source : page 25 https://cnofrance.org/wp-content/uploads/2021/05/2011_07_15-Normes-Usages-Indexed-Bonds.pdf

In [121]:
zc_df['real_rate'] = zc_df.apply(lambda x : ( (1 + x.zc_rate/100) / (1 + x.inf_swp_rate/100) -1) , axis=1)
#Discount factor taux ZC
zc_df['zc_df'] = zc_df.apply(lambda x : (1 + x.zc_rate/100)**x.alpha , axis=1)
#Discount factor taux réel
zc_df['rr_df'] = zc_df.apply(lambda x : (1 + x.real_rate/100)**x.alpha , axis=1)

<div align="left" style="border-bottom: 3px solid green; width: 20%;">
    <h6> Courbe Euribor 6M </h6>
</div>

Courbe Euribor 6M déduite de l'algorithme multicurve précédent

In [124]:
# FRN 6M based forward values base curve -- as per our multicurve algorithm result
eur_6mf_raw_df = pd.read_csv(DataDir+"\\Euribor_6M_Frwrd.csv", sep=',')
eur_6mf_raw_df.T

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
ID,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
Tenor,6M,7M,8M,9M,10M,11M,1Y,13M,15M,18M,...,6Y,7Y,8Y,9Y,10Y,12Y,15Y,20Y,25Y,30Y
Value,-0.312039,-0.35299,-0.360908,-0.361843,-0.359344,-0.357454,-0.3556,-0.355093,-0.352711,-0.346939,...,-0.31567,-0.283752,-0.246512,-0.2092,-0.172096,-0.090265,0.004395,0.076639,0.068646,0.033887


In [125]:
eur_6mf_df = dtf.add_dates(eur_6mf_raw_df, computation_date, computation_date, convention = "ACT/365")
#discount factor euribor 6M
eur_6mf_df['df'] = eur_6mf_df.apply(lambda x : (1 + x.Value/100)**x.alpha , axis=1)

Pour le pricing des bonds flotteurs on génère des 6 mois de taux déposit hisorique précédant la date computation_date

In [127]:
# generate Eur 6M deposit for last 6 months
histo_deposit_6m_df = pd.DataFrame({'Date' : [dtf.add_dmy(computation_date, -i, 0, 0 ) for i in range(zc_df.query("Tenor=='2Y'").day_diff.iloc[0]+1)]})
histo_deposit_6m_df.head()

,Date
0,2020-06-30
1,2020-06-29
2,2020-06-28
3,2020-06-27
4,2020-06-26


In [128]:
histo_deposit_6m_df['Value'] = np.random.uniform(1,3,histo_deposit_6m_df.shape[0])
histo_deposit_6m_df['Value_L1'] = histo_deposit_6m_df['Value'].shift(-1); 
histo_deposit_6m_df['Value_L2'] = histo_deposit_6m_df['Value'].shift(-2)
histo_deposit_6m_df.sort_values(by='Date', inplace=True) 
histo_deposit_6m_df.head()

,Date,Value,Value_L1,Value_L2
730,2018-07-01,2.063686,NaN,NaN
729,2018-07-02,2.206982,2.063686,NaN
728,2018-07-03,2.904235,2.206982,2.063686
727,2018-07-04,1.894643,2.904235,2.206982
726,2018-07-05,1.707884,1.894643,2.904235


In [129]:
#renvoyer les jours de la semaine, et le weeckend renvoyer vendredi
histo_deposit_6m_df['NewValue'] = histo_deposit_6m_df.apply(lambda x : [x.Value if x.Date.weekday()<=4 else x.Value_L1 if x.Date.weekday()==5 else x.Value_L2 ][0] , axis=1)
histo_deposit_6m_df['Value'] = histo_deposit_6m_df['NewValue'].fillna(histo_deposit_6m_df['Value'])
histo_deposit_6m_df.sort_values(by='Date', ascending=False, inplace=True) 
histo_deposit_6m_df.drop(columns=['Value_L1' ,'Value_L2', 'NewValue'], inplace=True)
#Nombre de jours entre la computation_date et la date
histo_deposit_6m_df['day_diff_lag'] = histo_deposit_6m_df.apply(lambda x : (computation_date - x.Date).days , axis=1)
histo_deposit_6m_df.head()

,Date,Value,day_diff_lag
0,2020-06-30,1.988632,0
1,2020-06-29,1.424596,1
2,2020-06-28,2.838114,2
3,2020-06-27,2.838114,3
4,2020-06-26,2.838114,4


<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>2.2 - Création de l'objet Bonds</h3>
</div>

In [131]:
class Bonds:
    def __init__(self, ISIN, first_accrual_date, first_coupon_date, maturity_date, TTM, coupon_rate, prices, freq, bond_type, spread, convention):
        #Identifiant du bond
        self.ISIN = ISIN
        #date de début et date de naissance des intérets
        self.first_accrual_date = first_accrual_date
        #date du premier coupon
        self.first_coupon_date = first_coupon_date
        #date de maturité
        self.maturity_date = maturity_date
        #temps de la computation_date à la maturité
        self.TTM = TTM
        #taux de coupon
        self.coupon_rate = coupon_rate
        #prix de l'obligation sans prendre en compte les intérêts courus
        self.clean_price = prices
        #coupon couru
        self.accr_coupon = np.nan
        #clean price + les intérêts courus depuis le dernier paiement du coupon jusqu'à la maturité
        self.dirty_price = np.nan
        #fréquence annuelle de l'obligation
        self.freq = freq
        #type d'obligation
        self.bond_type = bond_type
        #spread de coupon
        self.spread = spread
        #convention de l'obligation
        self.convention = convention
        #cash_flows de l'obligation où il y'aura un paiement (après la computation_date)
        self.cash_flows = bgf.derive_cash_flows(computation_date, maturity_date, freq, first_accrual_date, coupon_rate, spread, bond_type).query("upcoming_cf==1")
        #Z_spread de l'obligation
        self.z_spread = np.nan
        #yield to maturity
        self.ytm = np.nan
        #duration
        self.duration = np.nan
        #convexité
        self.convexite = np.nan


    def getAttribs(obj):
        DF_Attribs = pd.DataFrame({'attributes':vars(obj).keys()})
        return DF_Attribs.iloc[:,0]
    
    def getMethods(obj):
        DF_Attribs = pd.DataFrame({'attributes':vars(obj).keys()})
        DF_All = pd.DataFrame({'methods':[k if not(k.startswith('__')) else np.nan for k in dir(obj)]}).dropna()
        DF_All = DF_All[~DF_All.methods.isin(DF_Attribs.attributes)].reset_index(drop=True)
        return DF_All.iloc[:,0]

#méthode permettant de claculer  le coupons courrus   
    def get_accrued_cpn(self):
        if self.coupon_rate==0:
            accr_coupon=0
        else:
            #prochain coupon
            next_cf_dt = self.cash_flows.cf_dates.iloc[-1]
            # echéance précédente venue avant la prochaine échéance
            prev_theo_cf_dt = dtf.add_dmy(next_cf_dt, 0, -12/self.freq, 0)
            # Nombre de jours depuis le dernier coupon
            acrrual_num = (computation_date-prev_theo_cf_dt).days
            #Nombre de jours dans la période du coupon
            acrrual_denom_act = (next_cf_dt-prev_theo_cf_dt).days
            acrrual_denom_360 = 360
            # pour rappel le taux de coupon est annuel
            if self.convention=='ACT/ACT':
                accr_coupon = (self.coupon_rate/self.freq) * (acrrual_num/acrrual_denom_act)
            if self.convention=='ACT/360':
                accr_coupon = (self.coupon_rate/self.freq) * (acrrual_num/acrrual_denom_360)
        return accr_coupon
#Pricing d'une obligation       
    def zero_parallel_repricing(self, zero_shift):
        pv = np.sum(self.cash_flows.CFi * np.power(1+zero_shift, -self.cash_flows.Ti/365))
        return pv
#Pricing d'une obligation parallelement au taux réel (zc sans inflation)
    def full_parallel_repricing(self, shift):        
        pv = np.sum(self.cash_flows.CFi * np.power(1 + shift + self.cash_flows.zc_rr, -self.cash_flows.Ti/365))
        return pv

#implémentation de la méthode de newton_raphson  pour le ytm 
    def _newton_raphson_ytm(self, ytm):
        
        # Valeur actuelle des flux de trésorerie
        # pv = sum(cf / ((1 + ytm / 2) ** t) for t, cf in enumerate(cash_flows, start=1))
        pv = np.sum(self.cash_flows.CFi * np.power(1+ytm, -self.cash_flows.Ti/365))
        # Derivée de la valeur actuelle
        # derivative = sum(-t * cf / ((1 + ytm / 2) ** (t + 1)) for t, cf in enumerate(cash_flows, start=1))
        derivative = np.sum(-self.cash_flows.Ti/365 * self.cash_flows.CFi * np.power(1+ytm, -self.cash_flows.Ti/365 - 1))

        return ytm - (pv - self.dirty_price) / derivative

#calcul du Yield to maturity   
    def calcul_ytm(self):
        
        ytm = 0.05  # Valeur initiale pour le YTM
        for _ in range(10):  # Limite d'itérations
            ytm = self._newton_raphson_ytm(ytm)
        return ytm
        
#implémentation de la méthode de newton raphson pour le z_spread   
    def _newton_raphson_z_spread(self, z_spread):
        
        # Valeur actuelle des flux de trésorerie avec le Z-spread
        # pv = sum(cf / ((1 + z_spread / 2) ** t) for t, cf in enumerate(cash_flows, start=1))
        pv = np.sum(self.cash_flows.CFi * np.power(1+self.cash_flows.zc_rr+z_spread, -self.cash_flows.Ti/365))
        
        # Derivée de la valeur actuelle
        # derivative = sum(-t * cf / ((1 + z_spread / 2) ** (t + 1)) for t, cf in enumerate(cash_flows, start=1))
        derivative = np.sum(-self.cash_flows.Ti/365 * self.cash_flows.CFi * np.power(1+self.cash_flows.zc_rr+z_spread, -self.cash_flows.Ti/365 - 1))
        
        return z_spread - (pv - self.dirty_price) / derivative

#calcul du Z_spread  
    def calcul_z_spread(self):
        
        z_spread = 0.05  # Valeur initiale pour le Z-spread
        for _ in range(10):  # Limite d'itérations
            z_spread = self._newton_raphson_z_spread(z_spread)
        return z_spread

#Calculer la duration

    def compute_duration(self):
        duration = np.sum(self.cash_flows.Ti/365 * self.cash_flows.CFi * np.power(1 + self.ytm, -self.cash_flows.Ti/365 - 1))/self.dirty_price
        return duration
#Calculer la convexité   
    def compute_convexity(self):
        convexity = np.sum(self.cash_flows.Ti/365 * (self.cash_flows.Ti/365 + 1) * self.cash_flows.CFi * np.power(1 + self.ytm, -self.cash_flows.Ti/365 - 2))/self.dirty_price
        return convexity
#reprcing d'une obligation après une variation du ytm (shift)  
    def compute_delta_gamma(self, shift):
        PV01 = (1 -self.duration * shift + 0.5 * self.convexity * (shift**2) ) * self.dirty_price
        return PV01
     

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>2.3 - Calcul des z_spread et YTM </h3>
</div>

In [133]:
bond_df.head()

,bond_id,ISIN,accrual_per,TTM,initial_TTM,first_accrual_date,first_coupon_date,maturity_date,coupon_rate,prices,freq,bond_type,spread,convention
0,0,FR0038622365,23.400833,18.599167,42,1997-02-05,NaT,2039-02-03,0.000000,117.577045,0,zero,NaN,ACT/ACT
1,1,FR0021099549,12.327731,13.672269,26,2008-03-03,2009-03-02,2034-03-02,1.869007,114.508332,1,infla,NaN,ACT/ACT
2,2,FR0095538145,17.390829,2.609171,20,2003-02-10,2004-02-07,2023-02-07,1.172564,113.254871,1,fix,NaN,ACT/ACT
3,3,FR0034895483,4.198523,21.801477,26,2016-04-19,2017-04-18,2042-04-18,4.939976,109.258794,1,fix,NaN,ACT/ACT
4,4,FR0014637825,12.030527,12.969473,25,2008-06-19,2008-12-20,2033-06-20,NaN,108.213897,2,float,0.01915,ACT/360


In [134]:
cmp_start_time = time.time()
BondFactory={}
nb_bonds = bond_df.shape[0] #nombre d'obligations
for i in bond_df.index:
    start_time = time.time()
    print("Initialisation of bond N° "+str(i+1)+"/"+str(nb_bonds))
    #prendre la ligne i (caractéristiquesde l'obligation i)
    x = bond_df.loc[i,:].copy()
    #créer dans le dictionnaire la clé correspondant au code ISIN et la valeur correspondant à l'objet bonds
    BondFactory[x.ISIN] = Bonds(x.ISIN, x.first_accrual_date, x.first_coupon_date, x.maturity_date, x.TTM, x.coupon_rate, x.prices, x.freq, x.bond_type, x.spread, x.convention)

    if x.bond_type=='infla':
        #interpolation linéaire taux réel, trouver les r(ti) connaisant r(day_diff) où r cest le taux réel
        BondFactory[x.ISIN].cash_flows['zc_rr'] = np.interp( BondFactory[x.ISIN].cash_flows.Ti, zc_df.day_diff,zc_df.real_rate )
    if x.bond_type!='infla':
        #interpolation linéaire taux ZC, trouver les r(ti) connaisant r(day_diff) où r cest le taux ZC
        BondFactory[x.ISIN].cash_flows['zc_rr'] = np.interp( BondFactory[x.ISIN].cash_flows.Ti, zc_df.day_diff,zc_df.zc_rate/100 )
    if x.bond_type=='float':
        #les cash_flows de l'obligation
        tmp_float = BondFactory[x.ISIN].cash_flows.copy()
        #créer une variable qui à les dates des précédents cashflow lag 1 ? POURQUOI PAS SHIFT
        tmp_float['cf_dates_lag'] = tmp_float.apply(lambda y : dtf.adjust_date_bw(dtf.add_dmy(y.cf_dates, 0, -int(12/max(x.freq,1)), 0 )) , axis=1)
        #la différence de jours entre la date de calcul et le lag 1
        tmp_float['cf_daydiff_lag'] = tmp_float.apply(lambda y : (computation_date - y.cf_dates_lag).days , axis=1)
        tmp_float['eur_6m'] = np.nan
        if np.sum(tmp_float.cf_daydiff_lag>0)>0:
            #avant la date de calcul prendre les taux déposit
            tmp_float.loc[tmp_float.cf_daydiff_lag>0, 'eur_6m']  = Interps.SplineCubique(histo_deposit_6m_df['day_diff_lag'], histo_deposit_6m_df['Value'], tmp_float.loc[tmp_float.cf_daydiff_lag>0, 'cf_daydiff_lag']).y.values
        if np.sum(tmp_float.cf_daydiff_lag<=0)>0:
            #après la date de calcul prendre les taux euribor 6M de la courbe des taux
            tmp_float.loc[tmp_float.cf_daydiff_lag<=0, 'eur_6m']  = Interps.SplineCubique(eur_6mf_df['day_diff'], eur_6mf_df['df'], tmp_float.loc[tmp_float.cf_daydiff_lag<=0, 'cf_daydiff_lag'].abs()).y.values
        tmp_float.loc[:, 'coupon_rate']= np.where(tmp_float.eur_6m+tmp_float.spread<=0,0,tmp_float.eur_6m+tmp_float.spread)
        #ne prendre que les cashflows futurs
        BondFactory[x.ISIN].cash_flows.loc[:, 'eur_6m'] = tmp_float.query("upcoming_cf==1")[['eur_6m']]
        BondFactory[x.ISIN].cash_flows.loc[:, 'coupon_rate'] = tmp_float.query("upcoming_cf==1")[['coupon_rate']]
        BondFactory[x.ISIN].coupon_rate = tmp_float.query("upcoming_cf==1")[['coupon_rate']].iloc[-1].values[0]
        
    BondFactory[x.ISIN].cash_flows.loc[:, 'CFi'] =  BondFactory[x.ISIN].cash_flows.loc[:, 'coupon_rate']/max(x.freq,1) +  BondFactory[x.ISIN].cash_flows.loc[:, 'add_cf']
    BondFactory[x.ISIN].accr_coupon = BondFactory[x.ISIN].get_accrued_cpn()
    BondFactory[x.ISIN].dirty_price = BondFactory[x.ISIN].clean_price + BondFactory[x.ISIN].accr_coupon
    ellapsed_time = round(time.time() - start_time,2)
    print("Bond "+x.ISIN+" correctly initialised in --"+str(ellapsed_time)+" seconds. \n")

print("Overall initialisation performed in : "+str(round((time.time() - cmp_start_time),2))+" seconds. \n\n")





Initialisation of bond N° 1/10
Bond FR0038622365 correctly initialised in --0.02 seconds. 

Initialisation of bond N° 2/10
Bond FR0021099549 correctly initialised in --0.02 seconds. 

Initialisation of bond N° 3/10
Bond FR0095538145 correctly initialised in --0.01 seconds. 

Initialisation of bond N° 4/10
Bond FR0034895483 correctly initialised in --0.02 seconds. 

Initialisation of bond N° 5/10
Some values in SequenceToInterpolate appear to be out of the Curve_Xs range 
Bond FR0014637825 correctly initialised in --1.36 seconds. 

Initialisation of bond N° 6/10
Bond FR0086277509 correctly initialised in --0.01 seconds. 

Initialisation of bond N° 7/10
Bond FR0001819312 correctly initialised in --0.02 seconds. 

Initialisation of bond N° 8/10
Bond FR0003187954 correctly initialised in --0.02 seconds. 

Initialisation of bond N° 9/10
Bond FR0096173833 correctly initialised in --0.01 seconds. 

Initialisation of bond N° 10/10
Bond FR0072497946 correctly initialised in --0.01 seconds. 

Ov

In [135]:
#print("I) Attributes : \n"+ str(getAttribs(BondFactory['FR0038622365'])))
#print("\n \n II) Methods : \n"+ str(getMethods(BondFactory['FR0038622365'])))

Présentation des $Z_{spread}$ et des Yield to Maturity 

In [137]:

for i in list(BondFactory):
    start_time = time.time()
    print("Z-spread & YTM computation of "+i+" :")
    BondFactory[i].ytm = BondFactory[i].calcul_ytm()
    BondFactory[i].z_spread = BondFactory[i].calcul_z_spread()
    print("repricing Errors : \n"+
          " - Given dirty price = "+str(BondFactory[i].dirty_price)
          +"\n - For YTM = "+str(round(BondFactory[i].ytm,7))+" , we got repricing error = "+str(BondFactory[i].zero_parallel_repricing(BondFactory[i].ytm) - BondFactory[i].dirty_price)
          +"\n - For zspread = "+str(round(BondFactory[i].z_spread,7))+" , we got repricing error = "+str(BondFactory[i].full_parallel_repricing(BondFactory[i].z_spread) - BondFactory[i].dirty_price)
         )
    ellapsed_time = round(time.time() - start_time,2)
    print("Bond "+i+" correctly processed in --"+str(ellapsed_time)+" seconds. \n")


Z-spread & YTM computation of FR0038622365 :
repricing Errors : 
 - Given dirty price = 117.57704539134896
 - For YTM = -0.008664 , we got repricing error = -9.947598300641403e-14
 - For zspread = -0.0465143 , we got repricing error = -9.947598300641403e-14
Bond FR0038622365 correctly processed in --0.03 seconds. 

Z-spread & YTM computation of FR0021099549 :
repricing Errors : 
 - Given dirty price = 115.1227996037201
 - For YTM = 0.0074802 , we got repricing error = -1.1368683772161603e-13
 - For zspread = -0.0129623 , we got repricing error = 2.1316282072803006e-13
Bond FR0021099549 correctly processed in --0.02 seconds. 

Z-spread & YTM computation of FR0095538145 :
repricing Errors : 
 - Given dirty price = 113.7162081628926
 - For YTM = -0.0358298 , we got repricing error = 1.4210854715202004e-14
 - For zspread = -0.059482 , we got repricing error = 1.4210854715202004e-14
Bond FR0095538145 correctly processed in --0.03 seconds. 

Z-spread & YTM computation of FR0034895483 :
repri

<div align="center" style="border-bottom:solid  pink">
<h1> 3 - Full repricing et DeltaGamma</h1>
</div>

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>3.1 - DeltaGamma </h3>
</div>
 

**Formules pour la durée, la convexité et la variation du prix d’une obligation** \
**Convexité** \
La convexité d’une obligation mesure la courbure de la relation entre le prix et le rendement. Elle est donnée par :

$ C = \frac{1}{P} \sum_{t=1}^{n} \frac{t (t + 1) \cdot CF_t}{(1 + y)^{t + 2}} $


**Duration** \
La durée modifiée $D_m$ mesure la sensibilité du prix de l’obligation aux variations du rendement. Elle est calculée comme :

$D_m = \frac{1}{P} \sum_{t=1}^{n} \frac{t \cdot CF_t}{(1 + y)^{t + 1}} $

**La variation relative du prix d’une obligation** en fonction d’une variation du rendement peut être approximée à l’aide de la **duaration** et de la **convexité** comme suit :

$ \frac{\Delta P}{P} = -D \cdot \Delta y + \frac{1}{2} C \cdot (\Delta y)^2 $

ainsi :

$ Prix_{final} = (1-D \cdot \Delta y + \frac{1}{2} C \cdot (\Delta y)^2 )  \cdot Prix_{initial} $

Cette méthode est essentiellement utilisée pour un pricing parallèle (identique shift pour chaque $t_i$)

In [147]:

for i in list(BondFactory):
    start_time = time.time()
    print("Bond "+i+" :")
    BondFactory[i].duration = BondFactory[i].compute_duration()
    BondFactory[i].convexity = BondFactory[i].compute_convexity()
    ecb_shift = -25/10000
    print(" - Given dirty price = "+str(BondFactory[i].dirty_price)
          +"\n - YTM = "+str(round(BondFactory[i].ytm,7))
          +"\n - For senitivity purpose { TTM, mod_duration, convexity = "+str([BondFactory[i].TTM, BondFactory[i].duration, BondFactory[i].convexity])+"}"
          +"\n++++ Getting impact from 25Bps ("+str(ecb_shift)+") cut by ECB : "
          +"\n DeltaGamma(-25Bps) = "+str(BondFactory[i].compute_delta_gamma(ecb_shift))
         ) 
    ellapsed_time = round(time.time() - start_time,2)
    print("Bond "+i+" correctly processed in --"+str(ellapsed_time)+" seconds. \n")


Bond FR0038622365 :
 - Given dirty price = 117.57704539134896
 - YTM = -0.008664
 - For senitivity purpose { TTM, mod_duration, convexity = [18.599167142637626, 18.77084937181155, 371.27968704336405]}
++++ Getting impact from 25Bps (-0.0025) cut by ECB : 
 DeltaGamma(-25Bps) = 123.23101656483433
Bond FR0038622365 correctly processed in --0.0 seconds. 

Bond FR0021099549 :
 - Given dirty price = 115.1227996037201
 - YTM = 0.0074802
 - For senitivity purpose { TTM, mod_duration, convexity = [13.672268818729432, 12.160385238729356, 170.71125551303726]}
++++ Getting impact from 25Bps (-0.0025) cut by ECB : 
 DeltaGamma(-25Bps) = 118.68405845375868
Bond FR0021099549 correctly processed in --0.0 seconds. 

Bond FR0095538145 :
 - Given dirty price = 113.7162081628926
 - YTM = -0.0358298
 - For senitivity purpose { TTM, mod_duration, convexity = [2.6091710893216984, 2.6719340041690125, 9.966480392824254]}
++++ Getting impact from 25Bps (-0.0025) cut by ECB : 
 DeltaGamma(-25Bps) = 114.47935539

<div align="left" style="border-bottom: 3px solid #00008B; width: 50%;">
    <h3>3.2 - Full repricing </h3>
</div>

La méthode de calcul du z-spread peut être utilisée pour ajuster le prix d’une obligation dans deux contextes distincts : le **pricing parallèle** et le **pricing non parallèle**. 
___
**Pricing parallèle** 

Dans le cas d’un **pricing parallèle**, nous supposons un seul shift uniforme appliqué à tous les taux sans risque pour toutes les périodes. Ce shift, noté \( \text{shift} \), est ajouté au taux sans risque $ r(t_k) $ et au z-spread $ s $. Le prix de l’obligation, $ P_{\text{final}} $, est alors calculé comme suit :

$$ P_{\text{final}} = \sum_{k=t_i}^{N} \frac{C}{(1 + r(t_k) + s + \text{shift})^{\tau(t, t_k)}} + \frac{F}{(1 + r(t_N) + s + \text{shift})^{\tau(t, t_N)}} $$

Ce modèle est utile pour simuler un déplacement global de la courbe des taux, par exemple dans des tests de sensibilité.
___
**Pricing non parallèle**

Dans le cas d’un **pricing non parallèle**, nous permettons des shifts différents pour chaque période, reflétant des variations spécifiques aux différents facteurs de risque ou maturités. Le shift est alors un vecteur $ \text{shift} = (\text{shift}_1, \text{shift}_2, ..., \text{shift}_n) $, où chaque $ \text{shift}_k $ est appliqué à la période correspondante $ t_k $. Le prix de l’obligation devient :

$$ P_{\text{final}} = \sum_{k=t_i}^{N} \frac{C}{(1 + r(t_k) + s + \text{shift}_k)^{\tau(t, t_k)}} + \frac{F}{(1 + r(t_N) + s + \text{shift}_n)^{\tau(t, t_N)}} $$


In [149]:
basic_ecb_shift = (-25/10000)
dynamic_shift = zc_df.copy()
dynamic_shift['tenor_durations'] = np.where(dynamic_shift.day_diff<365,1, zc_df.day_diff/365).astype(int)
dynamic_shift['shift'] = basic_ecb_shift/dynamic_shift.tenor_durations
dynamic_shift = dynamic_shift[['day_diff','shift']]
dynamic_shift.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
day_diff,1.0000,30.0000,92.0000,183.0000,365.0000,730.00000,1095.000000,1826.0000,2556.000000,3653.00000,7307.000000,10957.000000,18262.00000
shift,-0.0025,-0.0025,-0.0025,-0.0025,-0.0025,-0.00125,-0.000833,-0.0005,-0.000357,-0.00025,-0.000125,-0.000083,-0.00005


In [153]:

for i in list(BondFactory):
    #nombre de cashflows
    n = BondFactory[i].cash_flows.query("upcoming_cf==1").shape[0]
    start_time = time.time()
    print("Bond"+i+" :")
    ecb_test_shift_parallel = -25/10000
    ecb_test_shift_non_parallel = np.interp(BondFactory[i].cash_flows.query("upcoming_cf==1").Ti, dynamic_shift['day_diff'] ,dynamic_shift['shift'] )
    print(" - Given dirty price = "+str(BondFactory[i].dirty_price)
          +"\n - zspread = "+str(round(BondFactory[i].z_spread,7))
          +"\n++++ Getting impact from 25Bps ("+str(ecb_test_shift_parallel)+") cut by ECB : "
          +"\n ParrallelShift-FullRepricing(-25Bps) = "+str(BondFactory[i].full_parallel_repricing(ecb_test_shift_parallel+BondFactory[i].z_spread)) 
          +"\n++++ Getting non parallel impact from  ("+str(ecb_test_shift_non_parallel[::-1])+") : " +"\n non ParrallelShift-FullRepricing(stress) = "
          +str(BondFactory[i].full_parallel_repricing(ecb_test_shift_non_parallel+BondFactory[i].z_spread))) 
    ellapsed_time = round(time.time() - start_time,2)
    print("Bond "+i+" correctly processed in --"+str(ellapsed_time)+" seconds. \n")


BondFR0038622365 :
 - Given dirty price = 117.57704539134896
 - zspread = -0.0465143
++++ Getting impact from 25Bps (-0.0025) cut by ECB : 
 ParrallelShift-FullRepricing(-25Bps) = 123.23341239391317
++++ Getting non parallel impact from  ([-0.00014262]) : 
 non ParrallelShift-FullRepricing(stress) = 117.89225000130192
Bond FR0038622365 correctly processed in --0.01 seconds. 

BondFR0021099549 :
 - Given dirty price = 115.1227996037201
 - zspread = -0.0129623
++++ Getting impact from 25Bps (-0.0025) cut by ECB : 
 ParrallelShift-FullRepricing(-25Bps) = 118.66739035775905
++++ Getting non parallel impact from  ([-0.0025     -0.00166096 -0.00097032 -0.00072116 -0.00055472 -0.00045205
 -0.00038063 -0.00033312 -0.00029747 -0.00026182 -0.00024165 -0.00022913
 -0.00021665 -0.00020416]) : 
 non ParrallelShift-FullRepricing(stress) = 115.43680312611946
Bond FR0021099549 correctly processed in --0.01 seconds. 

BondFR0095538145 :
 - Given dirty price = 113.7162081628926
 - zspread = -0.059482
++